# **<font color='crimson'>SQL. Window Functions</font>**

---

In [ ]:
###

In [ ]:
--1
SELECT
    t.product_id
    ,t.name
    ,t.price
    ,ROW_NUMBER() OVER w AS product_number
    ,RANK() OVER w AS product_rank
    ,DENSE_RANK() OVER w AS product_dense_rank
FROM
    products AS t
WINDOW w AS (ORDER BY t.price DESC);

In [ ]:
###

In [ ]:
--2
SELECT
    t.product_id
    ,t.name
    ,t.price
    ,MAX(t.price) OVER () AS max_price
    ,ROUND(t.price * 1. / MAX(t.price) OVER (), 2) AS share_of_max
FROM
    products AS t
ORDER BY
    t.price DESC
    ,t.product_id ASC;

In [ ]:
###

In [ ]:
--3
SELECT
    t.product_id
    ,t.name
    ,t.price
    ,MAX(t.price) OVER (ORDER BY t.price DESC) AS max_price
    ,MIN(t.price) OVER (ORDER BY t.price DESC) AS min_price
FROM
    products AS t
ORDER BY
    t.price DESC
    ,t.product_id ASC;

In [ ]:
###

In [ ]:
--4
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--формируем таблицу с общим числом заказов по дням
orders_per_day AS (
    SELECT
        t.creation_time::DATE AS date
        ,COUNT(t.order_id) AS orders_count
    FROM
        orders AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
    GROUP BY
        t.creation_time::DATE
    ORDER BY
        t.creation_time::DATE
)
--рассчитаем накопительную сумму числа заказов
SELECT
    t.date
    ,t.orders_count
    ,(SUM(t.orders_count) OVER (ORDER BY t.date ASC))::INT AS orders_cum_count
FROM
    orders_per_day AS t;

In [ ]:
###

In [ ]:
--5
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
)
--формируем таблицу с порядковым номером каждого заказа для каждого клиента
SELECT
    t.user_id
    ,t.order_id
    ,t.time
    ,ROW_NUMBER() OVER w AS order_number
FROM
    user_actions AS t
WHERE
    t.order_id NOT IN (
        SELECT t.order_id_cancel
        FROM orders_canceled AS t
    )
WINDOW w AS (
    PARTITION BY t.user_id
    ORDER BY t.time ASC)
LIMIT 1000;

In [ ]:
###

In [ ]:
--6
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
)
--формируем таблицу с порядковым номером каждого заказа для каждого клиента,
--временем, прошедшим между двумя разными заказами пользователя
SELECT
    t.user_id
    ,t.order_id
    ,t.time
    ,ROW_NUMBER() OVER w AS order_number
    ,LAG(t.time, 1) OVER w AS time_lag
    ,AGE(t.time, LAG(t.time, 1) OVER w) AS time_diff
FROM
    user_actions AS t
WHERE
    t.order_id NOT IN (
        SELECT t.order_id_cancel
        FROM orders_canceled AS t
    )
WINDOW w AS (
    PARTITION BY t.user_id
    ORDER BY t.time ASC)
ORDER BY
    t.user_id ASC
    ,t.order_id ASC
LIMIT 1000;

In [ ]:
###

In [ ]:
--7

Вариант решения, при котором рассчитывается время между последовательными парами заказов каждого пользоваталя и усредняется длительность между заказами одного пользователя между всеми парами его заказов.

In [ ]:
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--получим id клиентов, оформивших более двух заказов,
--исключив отмененные заказы
users_many_orders AS (
    SELECT
        t.user_id AS user_id_selected
    FROM
        user_actions AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
    GROUP BY
        t.user_id
    HAVING
        COUNT(t.order_id) > 1
),
--вычислим, сколько часов проходит
--между заказами каждого пользователя
hours_diff AS (
    SELECT
        t.user_id
        ,EXTRACT(
            EPOCH FROM
            t.time - LAG(t.time, 1) OVER w) / 3600
            AS hours_between_orders
    FROM
        user_actions AS t
    WHERE
        t.user_id IN (
        SELECT t.user_id_selected
        FROM users_many_orders AS t)
    WINDOW w AS (
        PARTITION BY t.user_id
        ORDER BY t.time ASC)
    ORDER BY
        t.user_id
)
--вычислим, сколько в среднем времени проходит
--между заказами каждого пользователя
SELECT
    t.user_id
    ,AVG(t.hours_between_orders)::INT AS hours_between_orders
FROM
    hours_diff AS t
GROUP BY
    t.user_id
LIMIT 1000;

Вариант решения, при котором находится самое раннее и самое позднее время заказа каждого пользователя, вычисляется интервал. Для нахождения среднего времени между заказами интервал делится на (число заказов - 1), - по количеству интервалов между заказами.

In [ ]:
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--посчитаем количество заказов каждого клиента,
--исключив отмененные заказы
users_many_orders AS (
    SELECT
        t.user_id
        ,t.order_id
        ,t.time
        ,COUNT(t.order_id) OVER (PARTITION BY t.user_id) AS orders_count
    FROM
        user_actions AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
),
--вычислим, самое ранее и самое позднее время
--заказов каждого пользователя,
--сделавшего 2 и более заказа
min_max_time AS (
    SELECT
        t.user_id
        ,t.orders_count
        ,MIN(t.time) OVER (PARTITION BY t.user_id) AS min_time
        ,MAX(t.time) OVER (PARTITION BY t.user_id) AS max_time
    FROM
        users_many_orders AS t
    WHERE
        t.orders_count > 1
)
--вычислим среднее время между заказами каждого пользователя,
--сделавшего два и более заказов
SELECT
    DISTINCT t.user_id
    ,ROUND(EXTRACT
        (EPOCH FROM t.max_time - t.min_time)
        / 3600 / (t.orders_count - 1))::INT
        AS hours_between_orders
FROM
    min_max_time AS t
ORDER BY
    t.user_id
LIMIT 1000;

In [ ]:
###

In [ ]:
--8
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--посчитаем количество уникальных заказов по дням
orders_per_day AS (
    SELECT
        t.creation_time::DATE AS date
        ,COUNT(t.order_id) AS orders_count
    FROM
        orders AS t
    WHERE t.order_id NOT IN (
        SELECT t.order_id_cancel
        FROM orders_canceled AS t)
    GROUP BY
        t.creation_time::DATE
    ORDER BY
        t.creation_time::DATE
)
--рассчитаем скользящее среднее числа заказов
SELECT
    t.date
    ,t.orders_count
    ,ROUND(AVG(t.orders_count) OVER w, 2) AS moving_avg
FROM
    orders_per_day AS t
WINDOW w AS (
    ORDER BY t.date
    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING)

In [ ]:
###

In [ ]:
--9

Вариант решения задачи с помощью конструкции CASE.

In [ ]:
--вычислим количество заказов,
--доставленных каждым курьером в сентябре 2022
WITH count_orders AS (
    SELECT
        DISTINCT t.courier_id
        ,COUNT(t.order_id) OVER w AS delivered_orders
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
        AND DATE_PART('year', t.time) = 2022
        AND DATE_PART('month', t.time) = 9
    WINDOW w AS (
        PARTITION BY t.courier_id)
    ORDER BY
        t.courier_id ASC
)
--рассчитаем среднее количество доставленных заказов
--по всем курьерам
SELECT
    t.courier_id
    ,t.delivered_orders
    ,ROUND(AVG(t.delivered_orders) OVER (), 2) AS avg_delivered_orders
    ,CASE
        WHEN t.delivered_orders > ROUND(AVG(t.delivered_orders) OVER (), 2) THEN 1
        ELSE 0
        END AS is_above_avg
FROM
    count_orders AS t
ORDER BY t.courier_id ASC;

Вариант решения задачи без контрукции CASE с помощью конвертации результатов логической операции (True / False) в тип данных INT.

In [ ]:
--вычислим количество заказов,
--доставленных каждым курьером в сентябре 2022
WITH count_orders AS (
    SELECT
        DISTINCT t.courier_id
        ,COUNT(t.order_id) OVER w AS delivered_orders
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
        AND DATE_PART('year', t.time) = 2022
        AND DATE_PART('month', t.time) = 9
    WINDOW w AS (
        PARTITION BY t.courier_id)
    ORDER BY
        t.courier_id ASC
)
--рассчитаем среднее количество доставленных заказов
--по всем курьерам
SELECT
    t.courier_id
    ,t.delivered_orders
    ,ROUND(AVG(t.delivered_orders) OVER (), 2) AS avg_delivered_orders
    ,(t.delivered_orders > ROUND(AVG(t.delivered_orders) OVER (), 2))::INT AS is_above_avg
FROM
    count_orders AS t
ORDER BY t.courier_id ASC;

In [ ]:
###

In [ ]:
--10
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--присвоим каждому заказу каждого пользовтаеля
--соответствующий статус 'Первый' или 'Повторный'
first_repeat_order AS (
    SELECT
        t.order_id
        ,t.time::DATE AS date
        ,CASE
            WHEN t.time = MIN(t.time) OVER (PARTITION BY t.user_id) THEN 'Первый'
            ELSE 'Повторный'
            END AS order_type
    FROM
        user_actions AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
    ORDER BY
        t.time::DATE ASC
        ,2 ASC
)
--посчитаем количество первых и повторных заказов
--по каждому дню
SELECT
    DISTINCT t.date
    ,t.order_type
    ,COUNT(t.order_type) OVER (PARTITION BY t.date, t.order_type) AS orders_count
FROM first_repeat_order AS t
ORDER BY
    t.date ASC
    ,t.order_type;

In [ ]:
###

In [ ]:
--11
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--присвоим каждому заказу каждого пользовтаеля
--соответствующий статус 'Первый' или 'Повторный'
first_repeat_order AS (
    SELECT
        t.order_id
        ,t.time::DATE AS date
        ,CASE
            WHEN t.time = MIN(t.time)
                OVER (PARTITION BY t.user_id) THEN 'Первый'
            ELSE 'Повторный'
            END AS order_type
    FROM
        user_actions AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
    ORDER BY
        t.time::DATE ASC
        ,2 ASC
),
--посчитаем количество первых и повторных заказов
--по каждому дню
order_types_per_day AS (
    SELECT
        DISTINCT t.date
        ,t.order_type
        ,COUNT(t.order_type) OVER (
            PARTITION BY t.date, t.order_type) AS orders_count
    FROM first_repeat_order AS t
    ORDER BY
        t.date ASC
        ,t.order_type
)
--для каждого дня посчитаем долю первых и повторных заказов
SELECT
    t.date
    ,t.order_type
    ,t.orders_count
    ,ROUND(
        t.orders_count / SUM(t.orders_count)
        OVER (PARTITION BY t.date), 2) As orders_share
FROM
    order_types_per_day AS t
ORDER BY
    t.date ASC
    ,t.order_type;

In [ ]:
###

In [ ]:
--12
SELECT
    t.product_id
    ,t.name
    ,t.price
    ,ROUND(AVG(t.price) OVER (), 2)
        AS avg_price
    ,ROUND(AVG(t.price) FILTER (
        WHERE t.price <> (
        SELECT MAX(t.price)
        FROM products AS t)) OVER (), 2)
        AS avg_price_filtered
FROM
    products AS t
ORDER BY
    t.price DESC
    ,t.product_id ASC;

In [ ]:
###

In [ ]:
--13
--вычисляем количество созданных и количество отмененных заказов
--каждым пользователем на каждую дату совершения действия
WITH create_cancel_orders AS (
SELECT
    t.user_id
    ,t.order_id
    ,t.action
    ,t.time
    ,COUNT(t.order_id)
        FILTER (WHERE t.action = 'create_order')
        OVER w AS created_orders
    ,COUNT(t.order_id)
        FILTER (WHERE t.action = 'cancel_order')
        OVER w AS canceled_orders
FROM
    user_actions AS t
WINDOW w AS (
    PARTITION BY t.user_id
    ORDER BY t.time ASC)
ORDER BY
    t.user_id ASC
    ,t.order_id ASC
    ,t.time ASC
)
--вычислим долю отмененных заказов в общем количестве заказов
SELECT
    t.user_id
    ,t.order_id
    ,t.action
    ,t.time
    ,t.created_orders
    ,t.canceled_orders
    ,ROUND(t.canceled_orders * 1. / t.created_orders, 2)
        AS cancel_rate
FROM
    create_cancel_orders AS t
LIMIT 1000;

In [ ]:
###

In [ ]:
--14
--вычислим количество заказов, доставленных каждым курьером
WITH number_of_orders AS (
    SELECT
        DISTINCT t.courier_id
        ,COUNT(t.order_id) OVER (
            PARTITION BY t.courier_id) As orders_count
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
    ORDER BY
        t.courier_id
),
--присвоим ранг курьерам в зависимости
--от количества доставленных заказов
rank_of_couriers AS
(
    SELECT
        t.courier_id
        ,t.orders_count
        ,ROW_NUMBER() OVER (
            ORDER BY
                t.orders_count DESC
                ,t.courier_id) AS courier_rank
    FROM
        number_of_orders AS t
    ORDER BY
        t.orders_count DESC
),
--вычислим ранг курьера, соответствующий нижней границе
--10% курьеров, доставивших больше всего заказов
ten_percent_bound AS (
    SELECT
        (MAX(t.courier_rank) * 0.1)::INT + 1 AS ten_percent_rank
    FROM
        rank_of_couriers AS t
)
--формируем итоговый список 10% курьеров, доставивших
--больше всего заказов
SELECT
    t.courier_id
    ,t.orders_count
    ,t.courier_rank
FROM
    rank_of_couriers AS t
WHERE
    t.courier_rank <= (
        SELECT t.ten_percent_rank
        FROM ten_percent_bound AS t)
ORDER BY
    t.courier_rank ASC;

In [ ]:
###

In [ ]:
--15
--получим самую позднюю дату действий курьеров
WITH last_date AS (
    SELECT MAX(t.time) AS max_time
    FROM courier_actions AS t
),
--вычислим отработанное каждым курьером время,
--а также количество доставленных заказов
subquery AS (
SELECT
    DISTINCT t.courier_id
    ,DATE_PART(
        'days',
        ((SELECT t.max_time FROM last_date AS t) -
            (MIN(t.time) OVER w)))::INT AS days_employed
    ,COUNT(t.order_id)
        FILTER (WHERE t.action = 'deliver_order')
        OVER w AS delivered_orders
FROM
    courier_actions AS t
WINDOW w AS (
    PARTITION BY t.courier_id)
)
--отфильтруем данные по курьерам, отработавшим
--более 10 дней
SELECT
    t.courier_id
    ,t.days_employed
    ,t.delivered_orders
FROM
    subquery AS t
WHERE
    t.days_employed >= 10
ORDER BY
    t.days_employed DESC
    ,t.courier_id ASC;

In [ ]:
###

In [ ]:
--16
--получим id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--'развернем' список продуктов в заказе из таблицы orders,
--заодно удалив отмененные заказы
unnest_table AS (
    SELECT
        t.order_id
        ,t.creation_time
        ,UNNEST(t.product_ids) AS product_id
    FROM
        orders AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM orders_canceled AS t)
),
--объединяем таблицы unnest_table и products
price_and_products AS (
    SELECT
        t.order_id
        ,t.creation_time
        ,t.product_id
        ,p.price
    FROM
        unnest_table AS t
        LEFT JOIN products AS p
            ON t.product_id = p.product_id
),
--вычислим стоимость каждого заказа
total_price_per_order AS (
    SELECT
        DISTINCT t.order_id
        ,t.creation_time
        ,SUM(t.price) OVER (PARTITION BY t.order_id) AS order_price
    FROM
        price_and_products AS t
),
--вычислим выручку за день
total_revenue_per_day AS (
SELECT
    t.order_id
    ,t.creation_time
    ,t.order_price
    ,SUM(t.order_price) OVER
         (PARTITION BY t.creation_time::DATE) AS daily_revenue
FROM
    total_price_per_order AS t
)
--вычислим долю стоимости заказа в выручке за день
SELECT
    t.order_id
    ,t.creation_time
    ,t.order_price
    ,t.daily_revenue
    ,ROUND(t.order_price / t.daily_revenue * 100, 3)
        AS percentage_of_daily_revenue
FROM
    total_revenue_per_day AS t
ORDER BY
    t.creation_time::DATE DESC
    ,percentage_of_daily_revenue DESC
    ,t.order_id ASC;

In [ ]:
###

In [ ]:
--17
--получаем id отмененных заказов
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_canceled
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--'разворачиваем' id товаров в каждом заказе,
--представленные в поле 'product_ids' таблицы orders;
--оставляем только те заказы, которые не были отменены
unnest_table AS (
    SELECT
        t.order_id
        ,t.creation_time::DATE AS date
        ,UNNEST(t.product_ids) AS product_id
    FROM
        orders AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_canceled
            FROM orders_canceled AS t)
),
--объединяем данные о каждом товаре в заказе с его ценой
product_and_price AS (
    SELECT
        t.order_id
        ,t.date
        ,t.product_id
        ,p.price
    FROM
        unnest_table AS t
        LEFT JOIN products AS p
            ON t.product_id = p.product_id
),
--вычислим ежедневную выручку сервиса
total_daily_sum AS (
    SELECT
        DISTINCT t.date
        ,SUM(t.price) OVER (PARTITION BY t.date) AS daily_revenue
    FROM
        product_and_price AS t
    ORDER BY
        t.date ASC
)
--вычислим абсолютный размер ежедневного прироста выручки
SELECT
    t.date
    ,ROUND(t.daily_revenue, 1) AS daily_revenue
    ,ROUND(COALESCE(
        t.daily_revenue - LAG(t.daily_revenue, 1) OVER (), 0), 1)
    AS revenue_growth_abs
    ,ROUND(COALESCE(
        (t.daily_revenue * 100. / LAG(t.daily_revenue, 1) OVER ()) - 100, 0), 1)
    AS revenue_growth_percentage
FROM
    total_daily_sum AS t
ORDER BY t.date ASC;

In [ ]:
###

In [ ]:
--18
--получим id заказов, которые не были отменены
WITH orders_canceled AS (
    SELECT
        t.order_id AS order_id_canceled
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--'разворачиваем' списки товаров каждого заказа
--отобрав только те заказы, которые не были отменены
unnest_table AS (
    SELECT
        t.order_id
        ,UNNEST(t.product_ids) AS product_id
    FROM
        orders AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id_canceled
            FROM orders_canceled AS t)
),
--объединяем сведения о товарах в каждом заказе
--с ценами каждого товара
product_and_price AS (
    SELECT
        t.order_id
        ,t.product_id
        ,p.price
    FROM
        unnest_table AS t
        LEFT JOIN products AS p
            ON t.product_id = p.product_id
),
--вычислим стоимость каждого заказа,
--результат уопрядочим по возрастанию суммы заказа
total_sum_per_order AS (
    SELECT
        DISTINCT t.order_id
        ,SUM(t.price) OVER (PARTITION BY t.order_id) AS order_price
    FROM
        product_and_price AS t
    ORDER BY
        SUM(t.price) OVER (PARTITION BY t.order_id) ASC
),
--присвоим номер каждому заказу
orders_ranks AS (
SELECT
    t.order_id
    ,t.order_price
    ,ROW_NUMBER() OVER (ORDER BY t.order_price ASC) AS order_rank
FROM
    total_sum_per_order AS t
),
--получим максимальный ранг заказа
maximum_rank AS (
    SELECT
        MAX(t.order_rank) AS max_rank
    FROM
        orders_ranks AS t
)
--вычислим медианную цену заказа
SELECT
    DISTINCT CASE
    WHEN (SELECT t.max_rank FROM maximum_rank AS t) % 2 = 1
        THEN (
            SELECT t.order_price
            FROM orders_ranks AS t
            WHERE t.order_rank = (
                SELECT t.max_rank FROM maximum_rank AS t) / 2 + 1)
    ELSE (
        SELECT SUM(t.order_price) * 1. / 2
        FROM orders_ranks AS t
        WHERE t.order_rank IN (
            (SELECT t.max_rank FROM maximum_rank AS t) / 2,
            (SELECT t.max_rank FROM maximum_rank AS t) / 2 + 1)
        )
    END AS median_price
FROM
    orders_ranks AS t;